# Repetition from days 1+2

We went through
 * Basic usage of python. Operators, data structures, flow control.
 * Manually and partly-automatic parsing of text files.
 * Some basic and not-so-basic plotting: double y-axes, logarithmic y axis, line graphs, bars and filled areas.

Questions
 * Anything that was (particularly) unclear?
 * Follow-up questions to what we went through?

# Day 3, before lunch: Time series analysis


Next step: Read stuff using pandas, making use of it's time series analysis and statistics functions.

But before we can start with that, we need to understand a little the basic data structure in pandas, a ``DataFrame``. Very briefly a ``DataFrame`` is a bit like a Table in an Excel/Calc spreadsheat:

 * It has columns and rows (hence two-dimensional structure).
 * Both columns and rows can have a header (a name).
 * ``DataFrames`` also have an index, a unique identifier for each entry (i.e. row) in the table.
 * Columns can have different data types

Let's see how we can build a ``DataFrame`` from the structures we already know.

In [ ]:
import pandas as pd

dfa = pd.DataFrame([3, 6, 9, 15, 24])
dfb = pd.DataFrame([(1,3), (1,6), (2,9), (3,15), (5,24)], columns=['Fib(1,1)', 'Fib(6,9)'])
dfc = pd.DataFrame({'Employee-ID': [1001, 1002, 1005], 'Names': ['Alice', 'Bob', 'Clarke'], 
                    'Income': [100000, 70000, 83000]})
dfd = dfc.set_index('Employee-ID')

#dfd

# We can use the column headers as attributes
#dfd.Income
type(dfd.Income)

Now that we have got a rough idea of what a DataFrame is, and how it might be useful, let's see what we can make use of that for analysing time series. We start by reading the rain time series from Bulken into pandas. pandas comes with it's own function to read text files, and one that is more versatile than the functions we used previously.

In [ ]:
df = pd.read_table(
    'd1s2/rr24_Bulken.txt', encoding='utf8', 
    header=13,
    skipfooter=12, engine='python',  # The file contains 12 footer lines, and skipfooter requires the python parsing engine
    sep='\s+',                       # Column separtion by one or more whitespace
    parse_dates=[1,], dayfirst=True, # Second column contains dates, in European format
    index_col=1,                     # Use the date column as index
)


#df.RR.plot()
df

We can apparently directly plot ``df.RR``, but what else can we do with it?

In [ ]:
dir(df.RR)

Many things available! We'll dive into a few of them later. 

First, a quick and easy first data analysis: Cumulative precipitation during that period.

In [ ]:
type(df.RR.cumsum())

Note: ``cumsum()`` returns a (time) series, so we can work with that in exactly the same way as ``df.RR``.

In [ ]:
df.RR.cumsum().plot()

Second example, find dates where the 24-hour precipitation exceeded 50 mm.

In [ ]:
df.index[df.RR > 50]

Worth taking some time to figure out in detail what happens here.

In [ ]:
type(df.RR > 50), (df.RR > 50).dtype, len(df.RR)

We're using a boolean time series to select dates.

We could also use any other (random) boolean time series of length 100.

In [ ]:
import numpy as np

# Note that we use the same index (the dates) as in df.RR
randombools = pd.Series(np.random.rand(100) >= 0.9, index=df.RR)
randombools.sum() # How many True values (=ones) are there in the time series?

In [ ]:
df.index[randombools]

Using the same mechanism, you can retrieve the entire data row instead of the dates

In [ ]:
df.loc[df.RR > 50]
# Or just
#df[df.RR > 50]

Nevertheless, you can still select rows by their integer index (starting the count at zero!).

In [ ]:
# Select the 46th and 59th entry in the DataFrame
df.iloc[[45,58]]

### Exercises 1: Monthly cumulative precipitation

The rain time series from Bulken contains three full months, August, September and October 2018.

Exercises: 
 * Print the cumulative precipitation for these three months to find out which month was wettest.
 * Plot the cumulative precipitation for these three months into the same figure. 

Hints: 
 * You will need the ``datetime`` objects from the ``datetime`` package to compare against the time series index.
 * You might need to convert the cumulative rain time series to a ``numpy`` array for the plotting.

In [ ]:
from datetime import datetime

month_names = {8: 'August', 9: 'September', 10: 'October'}
for month in [8, 9, 10]:
    #df_subset = df[df.index >= datetime(2018,month,1)]
    #df_subset = df_subset[df_subset.index < datetime(2018,month+1,1)]
    # alternative using logical_and
    df_subset = df[np.logical_and(df.index >= datetime(2018,month,1), df.index < datetime(2018,month+1,1))]
    print(f'Cumulative precipitation for {month_names[month]}: {df_subset.cumsum().RR.iloc[-1]:3.1f}')

In [ ]:
import matplotlib.pyplot as plt

for month in [8, 9, 10]:
    df_subset = df[np.logical_and(df.index >= datetime(2018,month,1), df.index < datetime(2018,month+1,1))]
    RR_per_month = np.array(df_subset.cumsum().RR)
    plt.plot(np.arange(len(RR_per_month)), RR_per_month, label=month_names[month])
plt.xlabel('Day of the month')
plt.ylabel('Cumulative precip. [mm]')
plt.legend()

## Comparing time series

So far we have only worked with one time series. Let's add a second to have some more analysis options to explore.

Unfortunately, the date format of ``rro_Bulken.txt`` is not recognised automatically by pandas, so we need to supply our custom conversion function.

In [ ]:
date_parser = lambda datestr: datetime.strptime(datestr, '%d%m%Y')

# Equivalent to
def date_parser(datestr):
    return datetime.strptime(datestr, '%d%m%Y')

This ``lambda`` is essentially just a shorthand for defining a function. We can use ``date_parser`` just as any other function.

In [ ]:
date_parser('23012019')

In [ ]:
df2 = pd.read_table('d1s2/rro_Bulken.txt', encoding='latin1', 
                   header=None, names=['Dato', 'Level', 'Discharge', 'p75', 'p50', 'p25'],
                                                    # Custom header information
                   comment='#',                     # Ignore lines starting with #
                   na_values=['----', ],            # Custom marker for missing values
                   sep='\s+',                       # Column separtion by one or more whitespace
                   parse_dates=[0,], date_parser=date_parser, 
                                                    # First column contains dates, custom format
                   index_col=0,                     # Use the date column as index
)

A quick sanity check to see whether we got what we expected.

In [ ]:
df2.Discharge.plot()

### Correlation analyses

Is river runoff correlated to precipitation?

In [ ]:
# Note the different time periods for df and df2!
df.RR.corr(df2.Discharge)

Seems like!

But what about the median discharge and precipitation?

In [ ]:
df2.p50.corr(df.RR)

More advance statistics will require the ``scipy.stats`` module. But to be able to use that module, we'll need to homogenise the two time series:
 * Identical index, i.e. dates
 * Remove NaNs
 
Once complete, we'll then make use of ``scipy.stats`` to estimate the significance of the above correlations and calculate lagged correlations between the two time series.

In [ ]:
# First combine into common data frame, using the same index

df_both = pd.concat([df, df2], axis=1)
df_both

In [ ]:
# Then extract the columns we are interested in, keeping only those rows where we have data in both

df_clean = df_both[['RR', 'Discharge']].dropna()
df_clean

Now we're finally ready to calculate the Pearson correlaton including it's significance. Documentation for the function is given here:

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr

In [ ]:
import scipy.stats

scipy.stats.pearsonr(df_clean.RR, df_clean.Discharge)

Cool, seems highly significant!

In [ ]:
df_clean = df_both[['RR', 'p50']].dropna()
scipy.stats.pearsonr(df_clean.RR, df_clean.p50)

While the correlation between median discharge and precipitation seems spurious.

With that clarified, onto lagged correlations.

### Exercise 2: Lagged correlations

We would expect that the river discharge is a delayed response of the precipitation. So we might expect that the maximum correlation is not instantaneous, but at a certain lead/lag.

Exercise: At which lead/lag in days does the correlation between precipitation reach it's maximum?
 * First, calculate the time lagged correlations and their significance for lags between -5 and 5 days. The ``pandas.Series.shift`` function might come in handy to do the actual shifting.
 * Second, visualise the results with a correlation lead/lag time series. Mark portions of the time series that are significant at the 99%-level.

First do the analysis.

In [ ]:
# We're using numpy arrays here in order to be able to filter the data by p-value later
tshifts = np.arange(-5,6)
corrcoefs = np.empty(tshifts.shape)
pvalues = np.empty(tshifts.shape)
for i, tshift in zip(range(len(tshifts)), tshifts):
    RRshift = df.RR.shift(tshift)
    dfs = pd.concat([RRshift, df2.Discharge], axis=1)
    dfs = dfs.dropna()
    corrcoefs[i], pvalues[i] = scipy.stats.pearsonr(dfs.RR, dfs.Discharge)

print(corrcoefs)
print(pvalues)

Then the plot.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(tshifts, corrcoefs, linewidth=1, color='C0')
plt.plot(tshifts[pvalues < 0.01], corrcoefs[pvalues < 0.01], linewidth=3, color='C0')
plt.xlabel('<- Discharge leads / Rain leads -> [days]')
plt.ylabel('Correlation coefficient')

plt.savefig('d2s1/ex2.pdf')

### Exercise 3: Smoothing time series

The river discharge might contain an integral of the precipitation over the preceeding days. Let's correlate smoothed precipitation with discharge, to see whether we can further increase the correlations. Use running means of 1-5 days centred on the given date in combination with lags between -5 and 5 days to find the maximum correlation.

The ``pandas.Series.rolling`` function might come in handy for calculating the running mean.

Again, first the analysis ...

In [ ]:
rmeans = np.arange(1,6)
tshifts = np.arange(-5,6)
corrcoefs = np.empty(rmeans.shape+tshifts.shape)
pvalues = np.empty(rmeans.shape+tshifts.shape)
for j, rmean in zip(range(len(rmeans)), rmeans):
    RRmean = df.RR.rolling(window=rmean, center=True).mean()
    for i, tshift in zip(range(len(tshifts)), tshifts):
        RRshift = RRmean.shift(tshift)
        dfs = pd.concat([RRshift, df2.Discharge], axis=1)
        dfs = dfs.dropna()
        corrcoefs[j,i], pvalues[j,i] = scipy.stats.pearsonr(dfs.RR, dfs.Discharge)

... then the plotting.

In [ ]:
for j, rmean in zip(range(len(rmeans)), rmeans):
    plt.plot(tshifts, corrcoefs[j], linewidth=1, color='C%d' % j)
    plt.plot(tshifts[pvalues[j,:] < 0.01], corrcoefs[j,pvalues[j,:] < 0.01], 
             linewidth=3, color='C%d' % j, label='%d-day mean' % rmean)
plt.xlabel('<- Discharge leads / Rain leads -> [days]')
plt.ylabel('Correlation coefficient')
plt.legend()
plt.savefig('d2s1/ex3.pdf')

### Exercise 4: Fitting linear model

For the combination of running mean and time lag that yields the maximum correlation create a linear model to estimate discharge from observed precipitation. 

We'll first do some preparatory work, before diving into the actual exercise. 

First step is to extract the time lag / smoothing that yielded the maximum correlation. 
 * The ``np.argmax`` yields the position of the maximum of the flattened the array. A flattened array is one-dimensional and contains the entries of the array in the order in which they appear in memory.
 * The ``np.unravel_index``-function then converts this index of the flattened array back to a multi-dimensional index appropriate for the original array.

In [ ]:
# Explain the following, make it NOT part of the assignment.
jmax, imax = np.unravel_index(np.argmax(corrcoefs), corrcoefs.shape)
print('Maximum correlation of %3.1f%% at rmean %d days and lag %d days' % (corrcoefs.max()*100, rmeans[jmax], tshifts[imax]))

Second step is to recreate the shifted running mean that gave the maximum correlation.

In [ ]:
RRmax = df.RR.rolling(window=rmeans[jmax], center=True).mean().shift(tshifts[imax])

As a preparation for the linear regression, we then collect all relevant ``Series`` in a new data frame, and rename the shifted / smoothed ``RR`` time series in the process.

In [ ]:
# Rename column to avoid having two columns named "RR", then concatenate to one dataframe
RRmax.rename('RRmax', inplace=True)
dfs = pd.concat([RRmax, df.RR, df2.Discharge], axis=1)
dfs_clean = dfs.dropna()

Finally we are ready for the actual exercise
 * Fit a linear model by linear regression. You can use ``scipy.stats.linregress`` do to the actual regression.
 * Create a new time series containing the river discharge from the precipitation observations
 * Evaluate the linear model fit using a ``plt.scatter``-plot comparing modelled versus observed discharge.
 * Evaluete the linear model by comparing the modelled and observed discharge time series.

Steps 1+2: Fitting the linear model and create the modelled discharge time series

In [ ]:
linmodel = scipy.stats.linregress(dfs_clean.RRmax, dfs_clean.Discharge)
estimate_Discharge = lambda RRm: linmodel.slope*RRm + linmodel.intercept
dfs['Discharge_linmodel'] = pd.Series(estimate_Discharge(dfs.RRmax), index=dfs.index)

Step 3: Evaluation using scatter plot

In [ ]:
plt.scatter(dfs.Discharge, dfs.Discharge_linmodel)
plt.plot([0, 500], [0, 500], 'k-', linewidth=1)
plt.xlabel('Observed discharge [m3/s]')
plt.ylabel('Estimated discharge from precipitation [m3/s]')
plt.savefig('d2s1/ex3_scatter.pdf')

Step 4: Evaluation comparing the time series directly

In [ ]:
dfs_clean = dfs.dropna()
dfs_clean.Discharge.plot(label='Observed')
dfs_clean.Discharge_linmodel.plot(label='Estimated')
plt.ylabel('Discharge [m3/s]')
plt.legend()
plt.savefig('d2s1/ex3_timeseries.pdf')